# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.

In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [ ]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

In [ ]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [ ]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether
google.generativeai.configure()

## An adversarial conversation between Chatbots.

### What if two chatbots get into a self-referential conversation that goes on a long time?  In my first test, 
### they eventually forgot the topic and ended up repeating polite nothings to each other.  In another test,
### they converged on a result and ended by exchanging nearly identical statements.

### Warning:  Think before you dial up the number of iterations too high.  Being a student, I don't know at what 
### point the chat becomes too costly or what models can do this without becoming overloaded.  Maybe Ed can advise if he sees this.

## Two chatbots edit an essay about cars.  One keeps trying to make it longer every time; the other keeps making it 
## shorter.


In [ ]:

# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"


gpt_system = "This is a description of a car; \
rephrase the description while adding one detail.  Don't include comments that aren't part of the car description."

claude_system = "This is a description of a car; \
repeat the description in slightly shorter form.  You may remove some details if desired.  Don't include comments that aren't part of the car description.  Maximum reply length 125 words."


gpt_messages = ["Hi there"]
claude_messages = ["Hi"]    


def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

reply = call_gpt()
print('\nGPT:  ', reply)

def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text


reply = call_claude()
print('\nGPT:  ', reply)

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")


### Here's the iterative loop.  Important change:  Unlike the original example, we don't repeat the entire conversation to make the input longer and longer.
### Instead, we use pop() to remove the oldest messages.

In [ ]:

for i in range(35):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    if len(gpt_messages) > 6:
        gpt_messages.pop(0)
        gpt_messages.pop(0)
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    if len(claude_messages) > 6:
        claude_messages.pop(0)
        claude_messages.pop(0)
    claude_messages.append(claude_next)

print('Done!')

